# Regularized Cox Models (LASSO & Elastic Net)
---
This notebook demonstrates how to fit **penalized Cox proportional hazards models** using `skglm`, including LASSO and Elastic Net regularization. We perform hyperparameter tuning via cross-validation and evaluate the model using the concordance index (C-index).

In [1]:
import types
import sys
from numbers import Real, Integral

# Create a fake module to emulate 'sklearn.utils._param_validation'
# (used by skglm in newer versions of scikit-learn, >=1.3)
param_validation = types.ModuleType("sklearn.utils._param_validation")

# Define a minimal replacement for Interval used in _parameter_constraints
class Interval:
    def __init__(self, dtype, left, right, closed="neither"):
        self.dtype = dtype
        self.left = left
        self.right = right
        self.closed = closed

# Define a minimal replacement for StrOptions used in _parameter_constraints
class StrOptions:
    def __init__(self, options):
        self.options = set(options)

# Add the custom classes to the fake module
param_validation.Interval = Interval
param_validation.StrOptions = StrOptions

# Inject the fake module into sys.modules before skglm is imported
# This prevents skglm from raising an ImportError if sklearn < 1.3
sys.modules["sklearn.utils._param_validation"] = param_validation

In [2]:
from tqdm import tqdm

In [3]:
# === Imports ===
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sksurv.metrics import concordance_index_censored
from lifelines.datasets import load_lung

# Load the built-in lung cancer dataset
df = pd.read_csv('./lung_dataset.csv')
# Set as binary variables
df["status"] = df["status"].apply(lambda x: 1 if x == 2 else 0)
df["sex"] = df["sex"].apply(lambda x: 1 if x == 2 else 0)

# Drop missing data
n_before = df.shape[0]
df = df.dropna()
print(f"Removed {n_before - df.shape[0]} observations containing NaN values.")

# Define features and target
X = df.drop(columns=["time", "status"])
y = df[["time", "status"]]  # Keep both as a DataFrame (not a tuple)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Combine X and y for lifelines
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Check shapes
print(f"Train: {train_df.shape}, Test: {test_df.shape}")

Removed 61 observations containing NaN values.
Train: (116, 11), Test: (51, 11)


## Cox LASSO (L1 Regularization)
The LASSO penalization encourages sparsity among coefficients. We tune the regularization strength `alpha` via grid search.

In [4]:
# === Cox LASSO (with cross-validation over penalizer) ===
penalties = np.logspace(-3, 1, 10)

best_cindex = -np.inf
best_penalty = None
best_model = None

for alpha in tqdm(penalties):
    # Define and fit LASSO-penalized Cox model
    cph = CoxPHFitter(penalizer=alpha, l1_ratio=1.0)
    cph.fit(train_df, duration_col="time", event_col="status")

    # Predict partial hazards on test set
    preds = -cph.predict_partial_hazard(test_df)

    # Compute C-index
    cindex = concordance_index(
        test_df["time"],
        preds,
        test_df["status"]
    )

    # Keep the best performing model
    if cindex > best_cindex:
        best_cindex = cindex
        best_penalty = alpha
        best_model = cph

print(f"Best penalizer (LASSO): {best_penalty:.4f}")
print(f"C-index (test): {best_cindex:.3f}")

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  5.77it/s]

Best penalizer (LASSO): 0.4642
C-index (test): 0.655


## Cox Elastic Net (L1 + L2 Regularization)
The Elastic Net mixes L1 and L2 penalties, balancing sparsity and stability. We tune both `alpha` and `l1_ratio`.

In [6]:
# === Cox Elastic Net ===
alphas = np.logspace(-3, 1, 10)     # penalizer strengths
l1_ratios = np.arange(0.1, 1.01, 0.1)        # mix between L1 and L2 penalties

best_cindex_en = -np.inf
best_params_en = None
best_model_en = None

# Grid search over (alpha, l1_ratio)
for alpha in tqdm(alphas, desc="Scanning penalizer values"):
    for l1_ratio in l1_ratios:
        cph = CoxPHFitter(penalizer=alpha, l1_ratio=l1_ratio)
        cph.fit(train_df, duration_col="time", event_col="status")

        preds = -cph.predict_partial_hazard(test_df)
        cindex = concordance_index(
            test_df["time"],
            preds,
            test_df["status"]
        )

        if cindex > best_cindex_en:
            best_cindex_en = cindex
            best_params_en = (alpha, l1_ratio)
            best_model_en = cph

print("\n=== Elastic Net optimization results ===")
print(f"Best penalizer (alpha): {best_params_en[0]:.4f}")
print(f"Best L1 ratio: {best_params_en[1]}")
print(f"Best C-index (test): {best_cindex_en:.3f}")

Scanning penalizer values: 100%|████████████████| 10/10 [00:15<00:00,  1.53s/it]


=== Elastic Net optimization results ===
Best penalizer (alpha): 0.4642
Best L1 ratio: 0.9
Best C-index (test): 0.655


## Discussion
- **LASSO** (`l1_ratio=1.0`) yields sparse solutions, selecting only the most relevant covariates.
- **Elastic Net** allows a compromise between feature selection (L1) and coefficient shrinkage (L2).
- The **C-index** measures the concordance between predicted risk scores and actual survival outcomes.
- Higher C-index indicates better discrimination ability.